<a href="https://colab.research.google.com/github/Jadrianp29/testreko/blob/main/DS_C7SC4_JesusAdrianAguilarPerez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modelo que distingue entre Perros y Gatos

Objetivos:

Diseñar modelos de redes neuronales profundas (deep learning), enfocándose en la clasificación de imágenes de acuerdo a categorías preestablecidas para resolver problemas con relevancia social, permitiendo generar valor en los diversos sectores.
Crear modelos de datos Deep Neural Networks (DNN) utilizando PyTorch basado en Python; seleccionando el modelo adecuado y analizando la exactitud del modelo, para cumplir lo mejor posible con los requerimientos de la tarea requerida.

Importar librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from PIL import Image
import torch.optim as optim
from torchvision import datasets, transforms
import os
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Importando carpeta

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Creamos Database asignamos un tamaño de 128*128 y hacemos data augmentation


In [4]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),
    transforms.ToTensor(),
])

Creamos directorio

In [5]:
DIR = datasets.ImageFolder("/content/drive/MyDrive/Tensorflow/DS_C7_SC4/catsvsdogs", transform=transform)

Se crea conjunto de entrenamiento y validacion

In [6]:
train_size = int(0.8 * len(DIR))
val_size = len(DIR) - train_size
train_dataset, val_dataset = random_split(DIR, [train_size, val_size])
val_loader = DataLoader(val_dataset, batch_size=32)

Funcion para Crear minibatches

In [7]:
x_train = []
y_train = []
for img, label in train_dataset:
    x_train.append(img.unsqueeze(0))
    y_train.append(label)

x_train = torch.cat(x_train)
y_train = torch.tensor(y_train)

In [8]:
def create_minibatches(x,y, mb_size):
  total_data = x.shape[0]
  idxs = np.arange(total_data)
  np.random.shuffle(idxs)
  x = x[idxs]
  y = y[idxs]
  return [(x[i:i+mb_size], y[i:i+mb_size]) for i in range(0, total_data, mb_size)]

In [17]:
minibatches = create_minibatches(x_train, y_train, mb_size=16)

Funcion para entrenar el modelo, Se utiliza Binary Cross Entropy (BCE) ya que solo son 2 clases

In [18]:
def train(model, optimizer, mb_size, epochs):
  for epoch in range(epochs):
    for xb, yb in create_minibatches(x_train, y_train, mb_size):
      xb = xb.to(device=device, dtype=torch.float32)
      model.train()
      scores = model(xb)
      yb = yb.to(device=device, dtype=torch.long)
      cost = F.cross_entropy(scores, yb)
      optimizer.zero_grad()
      cost.backward()
      optimizer.step()
    if epoch % 20 == 0:
      print(f"Epoch: {epoch}, Cost: {cost.item()},accuracy:{accuracy(model,val_loader)}")


Creacion de Red neuronal usando optimizador adam

In [19]:
model1 = nn.Sequential(
    nn.Conv2d(3, 20, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Conv2d(20, 40, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout2d(0.5),
    nn.MaxPool2d(2, 2),

    nn.Flatten(),
    nn.Linear(40 * 32 * 32, 100),
    nn.ReLU(),
    nn.Linear(100, 2)
)



In [20]:
modelo_final=model1.to(device=device)
adam=torch.optim.Adam(modelo_final.parameters(),lr=0.001)

Funcion para ver el accuracy de nuestro modelo

In [21]:
def accuracy(model,loader):
  num_correct = 0
  num_total = 0
  model.eval()
  model= model.to(device=device)


  with torch.no_grad():
     for(xb, yb) in loader:
       xb = xb.to(device=device, dtype=torch.float32)
       yb = yb.to(device=device, dtype=torch.long)


       scores = model(xb)
       _, pred= scores.max(dim=1)
       num_correct += (pred == yb.squeeze()).sum()
       num_total += pred.size(0)
     return float(num_correct) / num_total

In [22]:
train(modelo_final,adam,200,200)

Epoch: 0, Cost: 0.648689329624176,accuracy:0.7215007215007215
Epoch: 20, Cost: 0.5330218076705933,accuracy:0.7344877344877345
Epoch: 40, Cost: 0.17983798682689667,accuracy:0.6897546897546898
Epoch: 60, Cost: 0.04365674778819084,accuracy:0.7215007215007215
Epoch: 80, Cost: 0.031046763062477112,accuracy:0.7027417027417028
Epoch: 100, Cost: 0.017089765518903732,accuracy:0.7056277056277056
Epoch: 120, Cost: 0.01745196431875229,accuracy:0.7186147186147186
Epoch: 140, Cost: 0.010031916201114655,accuracy:0.70995670995671
Epoch: 160, Cost: 0.010847355239093304,accuracy:0.7070707070707071
Epoch: 180, Cost: 0.0025334935635328293,accuracy:0.7272727272727273
